In [ ]:
# Avery Schebell (ajs6av); Amshala Bharathan (ab5sz); Mianchen Cao (mc7sq)

In [ ]:
import requests
page = requests.get("https://dataquestio.github.io/web-scraping-pages/simple.html")
page

In [ ]:
page.status_code

In [ ]:
page.content

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content,'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
list(soup.children)

In [ ]:
[type(item) for item in list(soup.children)]

In [ ]:
html = list(soup.children)[2]

In [ ]:
list(html.children)

In [ ]:
body = list(html.children)[3]

In [ ]:
list(body.children)

In [ ]:
p = list(body.children)[1]

In [ ]:
p.get_text()

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

In [ ]:
soup.find_all('p')[0].get_text()

In [ ]:
soup.find('p')

In [ ]:
page = requests.get('https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html')
soup = BeautifulSoup(page.content,'html.parser')
soup

In [ ]:
soup.find_all('p',class_='outer-text')

In [ ]:
soup.find_all(class_='outer-text')

In [ ]:
soup.find_all(id='first')

In [ ]:
soup.select('div p')

In [ ]:
import pgeocode
print("Please enter a zipcode: ")
zipcode = input()
nomi = pgeocode.Nominatim('us')
# nomi.query_postal_code(zipcode)
longitude = str(nomi.query_postal_code(zipcode)['longitude'])
latitude = str(nomi.query_postal_code(zipcode)['latitude'])
print(longitude,latitude)


In [ ]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat="+latitude+"&lon="+longitude)
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())
print(forecast_items)

In [ ]:
period = tonight.find(class_='period-name').get_text()
short_desc = tonight.find(class_='short-desc').get_text()
temp=tonight.find(class_='temp').get_text()
print(period)
print(short_desc)
print(temp)

In [ ]:
img = tonight.find('img')
desc = img['title']
print(desc)

In [ ]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

In [ ]:
short_descs = [sd.get_text() for sd in seven_day.select('.tombstone-container .short-desc')]
temps = [t.get_text() for t in seven_day.select('.tombstone-container .temp')]
descs = [d['title'] for d in seven_day.select('.tombstone-container img')]
print(short_descs,temps,descs)


In [ ]:
#Seven day forecast
import pandas as pd
weather = pd.DataFrame({
    'period':periods,
    'short_desc':short_descs,
    'temp':temps,
    'desc':descs
})
weather

In [ ]:

temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather['temp_num']=temp_nums.astype('int')
temp_nums


In [ ]:
weather['temp_num'].mean()

In [ ]:
is_night=weather['temp'].str.contains("Low")
weather['is_night']=is_night
is_night

In [ ]:
weather

In [ ]:
#- humidity, windspeed, Dewpoint and Last Update Time
current_conditions = soup.find(id="current_conditions_detail")
# print(current_conditions)
soup = BeautifulSoup(page.content, 'html.parser')
humidity = soup.find_all('td')[1].get_text()
wind_speed = soup.find_all('td')[3].get_text()
dewpoint = soup.find_all('td')[7].get_text()
last_update_time = soup.find_all('td')[13].get_text().strip()
# print(last_update_time)
# print(humidity,wind_speed,dewpoint,last_update_time)
current_condition_display =pd.DataFrame([[humidity,wind_speed,dewpoint,last_update_time]], columns=['Humidity','Wind Speed','Dewpoint','Last Update Time'])
current_condition_display
# soup.find_all('p')
# forecast_items = seven_day.find_all(class_="text-right")
# tonight = forecast_items[0]
# print(tonight.prettify())
# print(forecast_items)

In [ ]:
import quandl
import pymongo
from pymongo import MongoClient

In [ ]:
client = MongoClient("mongodb://localhost:27017/")
data = weather.to_dict(orient = 'records')

In [ ]:
db = client['DS2002']

In [ ]:
db.lab5.insert_many(data)